## **🔰 Упрощенный код для Whisper [ver.2 Google Colab] 08-09-2023**

Полезные ссылки:

- **[Запуск whisper на Windows](https://www.itworks.hu/accelerating-speech-recognition-with-whisper-and-gpus-on-windows-11/)**

- **[Быстрые курсы по регулярным выражениям](https://regexlearn.com/ru/learn)**

In [ ]:
!ls -ahl

total 16K
drwxr-xr-x 1 root root 4.0K Sep  6 13:23 .
drwxr-xr-x 1 root root 4.0K Sep  8 15:07 ..
drwxr-xr-x 4 root root 4.0K Sep  6 13:22 .config
drwxr-xr-x 1 root root 4.0K Sep  6 13:23 sample_data


In [ ]:
#@markdown Проврека выделенной видео-карты (нужна T4 или выше)
!nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader,nounits


Fri Sep  8 15:10:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Установка yt-dlp
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.9 MB/s eta 0:00:00


In [ ]:
# Список ссылок для загрузки

urls_list = [
    "https://youtu.be/ih_B0-Y9gNE",
    "https://youtu.be/6DG5gzCXoRg?list=TLGGjOElj14_uCEyNTA4MjAyMw",
    "https://youtu.be/V1rcD1gDu9k",
    "https://www.youtube.com/embed/ih_B0-Y9gNE?iv_load_policy=3&autoplay=1&rel=0&version=3&loop=1&playlist=ih_B0-Y9gNE"
]

In [ ]:
# Функция очистки ссылок
import re

def clean_youtube_url(url: str) -> str:
    """
    Преобразует любую ссылку на видео YouTube в формат короткой ссылки (https://youtu.be/ID_ВИДЕО).

    Параметры:
        url (str): Исходная ссылка на видео на YouTube.

    Возвращает:
        str: Короткая ссылка на видео или None, если исходная ссылка не соответствует формату YouTube.

    Пример:
        >>> clean_youtube_url("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        "https://youtu.be/dQw4w9WgXcQ"
    """

    # Регулярное выражение для поиска идентификаторов видео YouTube:
    # 1. (?:https?:\/\/)? - необязательный протокол (http или https).
    # 2. (?:www\.)? - необязательный префикс "www".
    # 3. (?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/) - паттерн для длинных (стандартных и embed) и коротких ссылок YouTube.
    # 4. ([a-zA-Z0-9_-]{11}) - идентификатор видео, состоящий из 11 символов.
    pattern = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/)([a-zA-Z0-9_-]{11})"

    # Поиск совпадения с помощью регулярного выражения
    match = re.search(pattern, url)
    if match:
        # Если найдено совпадение, извлекаем идентификатор видео
        video_id = match.group(1)
        return f"https://youtu.be/{video_id}"
    else:
        return None

In [ ]:
# Создаем список "очищенных" коротких ссылок на видео YouTube.
# Все недопустимые или неподходящие ссылки будут проигнорированы.
cleaned_urls = set(filter(None, map(clean_youtube_url, urls_list)))

# Выводим результат
print(cleaned_urls)

{'https://youtu.be/ih_B0-Y9gNE', 'https://youtu.be/6DG5gzCXoRg', 'https://youtu.be/V1rcD1gDu9k'}


In [ ]:
# Функция загрузки видео в формате m4a (аудиофайл) с YouTube в директоррию /content/videos/
import subprocess

def download_video(url: str) -> None:
    """
    Загружает видео с YouTube в формате m4a (аудиофайл) и сохраняет в директории /content/audios/.

    Параметры:
        url (str): Ссылка на видео на YouTube.

    Пример:
        >>> download_video("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        ...
    """

    # Команда для yt-dlp, которая:
    # 1. Использует опцию "-x" для извлечения аудио.
    # 2. Устанавливает формат аудио в "m4a".
    # 3. Определяет путь для сохранения файла.
    cmd = [
        "yt-dlp",
        "-x",
        "--audio-format", "m4a",
        "-o", "/content/audios/%(title)s.%(ext)s",
        url
    ]

    try:
        # Инициализация подпроцесса с заданной командой.
        # stdout=subprocess.PIPE позволяет читать вывод в реальном времени.
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Чтение вывода команды в реальном времени и его вывод на экран.
        for line in process.stdout:
            print(line.strip())

        # Ожидание завершения подпроцесса и получение кода завершения.
        return_code = process.wait()

        # Если процесс завершился с ошибкой (не нулевой код завершения), генерируем исключение.
        if return_code != 0:
            raise subprocess.CalledProcessError(return_code, cmd)

    # Обработка исключений при выполнении команды.
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при обработке ссылки {url}:")
        print(str(e))

In [ ]:
# Перебор каждой очищенной ссылки из списка cleaned_urls.
# Для каждой ссылки будет вызвана функция download_video,
# которая загрузит видео в формате m4a и сохранит его в директории /content/audios/.
for url in cleaned_urls:
    download_video(url)

[youtube] Extracting URL: https://youtu.be/ih_B0-Y9gNE
[youtube] ih_B0-Y9gNE: Downloading webpage
[youtube] ih_B0-Y9gNE: Downloading ios player API JSON
[youtube] ih_B0-Y9gNE: Downloading android player API JSON
[youtube] ih_B0-Y9gNE: Downloading player 7ee36b0e
[youtube] ih_B0-Y9gNE: Downloading m3u8 information
[info] ih_B0-Y9gNE: Downloading 1 format(s): 251
[download] Destination: /content/audios/Новая Kia Cerato： стало круто？ ⧸ Киа Церато 2021 тест и обзор.webm

[download]   0.0% of   12.02MiB at  600.22KiB/s ETA 00:20
[download]   0.0% of   12.02MiB at    1.33MiB/s ETA 00:09
[download]   0.1% of   12.02MiB at    2.48MiB/s ETA 00:04
[download]   0.1% of   12.02MiB at    4.42MiB/s ETA 00:02
[download]   0.3% of   12.02MiB at    3.10MiB/s ETA 00:03
[download]   0.5% of   12.02MiB at    3.99MiB/s ETA 00:02
[download]   1.0% of   12.02MiB at    5.07MiB/s ETA 00:02
[download]   2.1% of   12.02MiB at    7.11MiB/s ETA 00:01
[download]   4.2% of   12.02MiB at   11.07MiB/s ETA 00:01
[downl

In [ ]:
# Установка whisper
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.4 MB/s eta 0:00:00


In [ ]:
import os
from typing import List

def _construct_whisper_command(input_path: str, output_dir: str) -> List[str]:
    """
    Формирование команды для программы whisper.

    Args:
    - input_path (str): Путь к исходному аудиофайлу.
    - output_dir (str): Путь к директории, где сохранить результаты транскрибации.

    Returns:
    - List[str]: Список аргументов для команды whisper.

    Команда whisper используется для автоматической транскрибации аудиозаписей.
    В данной функции мы формируем список аргументов для этой команды:
    1. `--model large-v2`: использование улучшенной большой модели (версии 2) для транскрибации.
    2. `--language ru`: указание языка речи на русском.
    3. `--device cuda`: использование графического процессора (GPU) для ускорения транскрибации.
    4. `--output_format txt`: формат вывода результатов транскрибации в текстовом файле.
    """
    return [
        'whisper',
        input_path,
        '--model', "large-v2",
        '--language', 'ru',
        '--device', 'cuda',
        '--output_format', 'all',
        '--output_dir', output_dir
    ]


def transcribe_audio_files(input_directory: str, output_directory: str) -> None:
    """
    Транскрибирование всех аудиофайлов из указанной директории с помощью whisper.

    Args:
    - input_directory (str): Директория с исходными аудиофайлами.
    - output_directory (str): Директория для сохранения результатов транскрибации.

    Для каждого файла из `input_directory` запускается процесс транскрибации.
    Результаты сохраняются в поддиректории `output_directory`, где каждая поддиректория соответствует одному аудиофайлу.
    """

    # Проверка наличия выходной директории и её создание при отсутствии
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Формирование списка аудиофайлов с расширением .m4a
    files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f)) and f.endswith('.m4a')]

    # Для каждого аудиофайла:
    for file in files:
        input_path = os.path.join(input_directory, file)

        # Имя поддиректории формируется на основе имени файла без расширения
        subdir_name = os.path.splitext(file)[0]
        subdir_path = os.path.join(output_directory, subdir_name)

        # Информирование пользователя о текущем файле
        print(f"Транскрибирование файла: {file}...")

        # Формирование команды для whisper
        cmd = _construct_whisper_command(input_path, subdir_path)

        # Запуск процесса транскрибации и вывод результатов в реальном времени
        with subprocess.Popen(cmd, stdout=subprocess.PIPE, text=True) as process:
            for line in iter(process.stdout.readline, ''):
                print(line, end='')  # Вывод строки в реальном времени
            print(f"\nТранскрибирование файла {file} завершено.")


In [ ]:
transcribe_audio_files('/content/audios', '/content/out')

Транскрибирование файла: Новый KIA Ceed ⧸ SW 2022： БЕЗ ПАЛКИ С БАЛКОЙ. Тест-Драйв.m4a...
[00:00.000 --> 00:04.000]  Привет, друзья! У KIA Ceed обновился логотип.
[00:04.000 --> 00:08.000]  По этому поводу мы собрали чемоданы и приехали в Ростов-на-Дону
[00:08.000 --> 00:12.000]  для того, чтобы снять эту машину с новыми буквами.
[00:12.000 --> 00:16.000]  Нет, под такой повод, ребята, нас сорвать с места довольно трудно.
[00:16.000 --> 00:23.000]  А вот то, что здесь под капотом новый турбомотор 1.5 мощностью 150 лошадиных сил,
[00:23.000 --> 00:26.000]  то, что внутри комплекс телематических систем,
[00:26.000 --> 00:31.000]  как у BMW и Volvo, и даже покруче в некоторых местах,
[00:31.000 --> 00:36.000]  то, что здесь фонари, от которых сносят крышу, светодиодные фары,
[00:36.000 --> 00:40.000]  я вам сейчас все расскажу, и бамперы тоже другие.
[00:40.000 --> 00:42.000]  Вот это все в комплексе.
[00:42.000 --> 00:46.000]  Мы готовы, да, поснимать для вас в Ростове-на-Дону.
[00:46.000

In [ ]:
!ls -ahl /content/audios

total 44M
drwxr-xr-x 2 root root 4.0K Sep  8 15:13  .
drwxr-xr-x 1 root root 4.0K Sep  8 15:13  ..
-rw-r--r-- 1 root root 2.2M Oct 25  2021 'Kia. Двигайся. Вдохновляйся.m4a'
-rw-r--r-- 1 root root  16M Sep 20  2021 'Новая Kia Cerato： стало круто？ ⧸ Киа Церато 2021 тест и обзор.m4a'
-rw-r--r-- 1 root root  26M Sep 16  2022 'Новый KIA Ceed ⧸ SW 2022： БЕЗ ПАЛКИ С БАЛКОЙ. Тест-Драйв.m4a'


In [ ]:
!ls -ahl /content/out

total 20K
drwxr-xr-x 5 root root 4.0K Sep  8 15:32  .
drwxr-xr-x 1 root root 4.0K Sep  8 15:13  ..
drwxr-xr-x 2 root root 4.0K Sep  8 15:33 'Kia. Двигайся. Вдохновляйся'
drwxr-xr-x 2 root root 4.0K Sep  8 15:31 'Новая Kia Cerato： стало круто？ ⧸ Киа Церато 2021 тест и обзор'
drwxr-xr-x 2 root root 4.0K Sep  8 15:23 'Новый KIA Ceed ⧸ SW 2022： БЕЗ ПАЛКИ С БАЛКОЙ. Тест-Драйв'
